In [ ]:
pip install openseespy

## **Modeling this Turbine**
**65-kW Wind Turbine courtesy of UCSD**
<img src='Research\Turbine.PNG' width="300" height="300">

In [ ]:
# following example on http://opensees.berkeley.edu/wiki/index.php/Elastic_Frame_Example
# running dynamic ground motion

%matplotlib notebook

# import OpenSees and libraries
from openseespy.opensees import *
from openseespy.postprocessing.Get_Rendering import *

# numerical and plotting tools
import numpy as np
import matplotlib.pyplot as plt
import math
from math import sqrt
import pandas as pd
import csv

# system commands
import os, os.path
import glob
import shutil

# -------------------------------
#       Generate Model
# -------------------------------

# remove existing model
wipe()

# remove existing results
# explanation here: https://stackoverflow.com/a/31989328
def remove_thing(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

def empty_directory(path):
    for i in glob.glob(os.path.join(path, '*')):
        remove_thing(i)

empty_directory('modes')
empty_directory('output')

# ---------------------------------------
#   Generate model and static analysis
# ---------------------------------------

# set modelbuilder
model('basic', '-ndm', 3, '-ndf', 6)

# units: in, kip, s
# dimensions
ft = 12.0
inch = 1.0
g = 386.1 #in/s^2
kip = 1.0
ksi = kip/(inch**2)
s = 1

# material properties
Es = 29000*ksi
Gs = 11500*ksi

# -------------------------------
#       Create Nodes
# -------------------------------
# command: node(nodeID, x-coord, y-coord, z-coord)
# command: node(nodeTag, *crds, '-ndf', ndf, '-mass', *mass, '-disp', *disp, '-vel', *vel, '-accel', *accel)
# Note: Ian Prowell Dissertation: Split into 30 beam-column elements

# specify number of elements of each portion of the tower
eldiv = 6

# Defining the first node
nodeTag = 1
node(nodeTag, 0.0, 0.0, 0.0)

# Heights of tower sections
hbot = 238.2*inch
hbottap = 74.8*inch
hmid = 237.2*inch
hmidtap = 74.8*inch
htop = 263*inch # to reach the top of the nacelle 238 og 

# Lower tower section (Split into eldiv elements)
for j in range(eldiv):
    nodeTag += 1
    h = (hbot/eldiv)*(j + 1)
    node(nodeTag, 0.0, 0.0, h)
    
# Tapered lower section (Split into eldiv elements)
for j in range(eldiv):
    nodeTag += 1
    h = hbot + (hbottap/eldiv)*(j + 1)
    node(nodeTag, 0.0, 0.0, h)

# Middle tower section (Split into eldiv elements)
for j in range(eldiv):
    nodeTag += 1
    h = hbot + hbottap + (hmid/eldiv)*(j + 1)
    node(nodeTag, 0.0, 0.0, h)

# Tapered middle section (Split into eldiv elements)
for j in range(eldiv):
    nodeTag += 1
    h = hbot + hbottap + hmid + (hmidtap/eldiv)*(j + 1)
    node(nodeTag, 0.0, 0.0, h)
    
# Top tower section (Split into eldiv elements)
for j in range(eldiv):
    nodeTag += 1
    h = hbot + hbottap + hmid + hmidtap + (htop/eldiv)*(j + 1)
    node(nodeTag, 0.0, 0.0, h)
    
# restraints
# command: fix(nodeID, DOF1, DOF2, DOF3, DOF4, DOF5, DOF6) 0 = free, 1 = fixed
# Use a fixed connection for now
fix(1, 1, 1, 1, 1, 1, 1)

# geometric transformation for beam-columns
# command: geomTransf('Type', TransfTag)
# see https://opensees.berkeley.edu/wiki/index.php/Linear_Transformation 
geomTransf('PDelta', 1, 0, 1, 0) #columns

# -------------------------------
#       Define Elements
# -------------------------------

# initialize lists
D = [] # diameters in
A = [] # areas in^2
I = [] # area moment of inertia in^4
J = [] # polar moment of inertia in^4
dM = [] # distributed masses kip*s^2/in
V = [] # volumes in^3
H = [] # height of each element in
t = 0.21*inch # thickness is constant


# Determining the diameters, areas, area moment of inertias, and polar moments of each discretized tower segment
# diameters of tower
dbot = 78.7*inch
dmid = 62.9*inch
dtop = 47.2*inch

index = 0

# Lower tower section (Split into eldiv elemenets)
for j in range(eldiv):
    H.append(hbot/eldiv)
    D.append(dbot)
    A.append(((D[index])**2 - (D[index] - (2*t))**2)*math.pi/4)
    I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
    J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
    index += 1

# Tapered lower section (Split into eldiv elements)
for j in range(eldiv):
    H.append(hbottap/eldiv)
    d = ((j + 1)/(2*eldiv))*(dmid-dbot) + dbot # interpolating for the diameter based on eldiv
    D.append(d)
    A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
    I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
    J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
    index += 1

# Middle tower section (Split into eldiv elements)
for j in range(eldiv):
    H.append(hmid/eldiv)
    D.append(dmid)
    A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
    I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
    J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
    index += 1

# Tapered middle section (Split into eldiv elements)
for j in range(eldiv):
    H.append(hmidtap/eldiv)
    d = ((j + 1)/(2*eldiv))*(dtop-dmid) + dmid # interpolating for the diameter based on eldiv
    D.append(d)
    A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
    I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
    J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
    index += 1
    
# Top tower section
for j in range(eldiv):
    H.append(htop/eldiv)
    D.append(dtop)
    A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
    I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
    J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
    index += 1
    
# Determining volumes of tower segments
index = 0

# Lower tower section (Split into eldiv elemenets)
for j in range(eldiv):
    v = math.pi*(hbot/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
    V.append(v)
    index += 1
    
# Tapered lower section (Split into eldiv elements)
for j in range(eldiv):
    dlower = ((dmid-dbot)*j)/(eldiv) + dbot
    dhigher = ((dmid-dbot)*(j+1))/(eldiv) + dbot
    v = math.pi*(hbottap/eldiv)*((dlower**2 +dlower*dhigher + dhigher**2)-((dlower-2*t)**2 + (dlower-2*t)*(dhigher-2*t) + (dhigher-2*t)**2))/12
    V.append(v)
    index += 1    

# Middle tower section (Split into eldiv elements)
for j in range(eldiv):
    v = math.pi*(hmid/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
    V.append(v)
    index += 1
    
# Tapered middle section (Split into eldiv elements)
for j in range(eldiv):
    dlower = ((dtop-dmid)*j)/(eldiv) + dmid
    dhigher = ((dtop-dmid)*(j+1))/(eldiv) + dmid
    v = math.pi*(hmidtap/eldiv)*((dlower**2 +dlower*dhigher + dhigher**2)-((dlower-2*t)**2 +(dlower-2*t)*(dhigher-2*t) + (dhigher-2*t)**2))/12
    V.append(v)
    index += 1    

# Top tower section (Split into eldiv elements)
for j in range(eldiv):
    v = math.pi*(htop/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
    V.append(v)
    index += 1


mtotal = 14.101*kip/g
vtotal = sum(V)
htotal = hbot + hbottap + hmid + hmidtap + htop

# Determining distributed masses of tower segments
# Entire tower section (Split into 5*eldiv elemenets)
for j in range(len(H)):
    distm = (mtotal*V[j])/(vtotal*H[j]) # mass/height constant
    dM.append(distm)
    
# Defining the elements and determining areas, area moment of inertias, and polar inertias for each section
# Cylindrical sections => Ix = Iy 
# Cylindrical sections => J = Iz = 2*Ix = (D^4-(D-t)^4)*(pi/32) 

# Assigning element properties
# create elastic beam-column elements - 
# command: element('elasticBeamColumn', eleTag, *eleNodes, Area, E_mod, G_mod, Jxx, Iy, Iz, transfTag, <'-mass', mass>, <'-cMass'>)
# define the columns  
for j in range(len(H)):
    element('elasticBeamColumn', j+1, j+1, j+2, A[j], Es, Gs, J[j], I[j], I[j], 1, '-mass', dM[j])

# Uncomment to see how the turbine twists
#node(nodeTag +1, 0.0, 10.0*ft, h)
#geomTransf('Linear', 2, 0, 0, 1) #columns
#element('elasticBeamColumn', len(H) + 1, len(H) + 1, len(H) + 2, A[len(H)-2], Es, Gs, J[len(H)-2], I[len(H)-2], I[len(H)-2], 2, '-mass', dM[len(H)-2])
    
# assign additional masses
# masses only act at nodes that have DoF
# command: mass(nodeID, dx, dy, dz, r@x, r@y, r@z)
# rotational inertia for a point mass: I = mr^2
whub = 9.4*kip
mhub = whub/g #kip*s^2/in
### to-do: the box is slightly higher than the last node of the tower, should we add another node? how would it be connected to the rest of the tower?
hhub = 888*inch # height of hub
#node(len(H) + 2, 0.0, 0.0, hhub) # CHANGE the top height it this is used
#rigidLink('beam', len(H)+1, len(H)+2)
mass(len(H) + 1, mhub, mhub, mhub, 0.0, 0.0, 0.0)

# -------------------------------
#       Set up Static analysis
# -------------------------------

# # create TimeSeries 
# timeSeries("Linear", 1)

# # create plain load pattern
# # command pattern('Plain', tag, timeSeriesTag)
# pattern('Plain', 1, 1)

# # Distributed element load of segments
# for j in range(len(H)-1):
#     eleLoad('-ele', j+1, j+1, j+2, '-type', '-beamUniform', -dM[j]*g, 0.0, 0.0)

# # Concentrated load of nacelle and hub
# load(len(H) + 1, 0.0, -whub, 0.0, 0.0, 0.0, 0.0)

# # ------------------------------
# # Start of analysis generation
# # ------------------------------

# # create system of equation (SOE)
# system("BandSPD")
# # create DOF numberer
# numberer("RCM")
# # create constraint handler
# constraints("Plain")
# # create integration scheme (steps of 1.0)
# integrator("LoadControl", 1.0)
# # create solution algorithm
# algorithm("Linear")
# # create analysis object
# analysis("Static")
# # perform the analysis
# analyze(1)


# # set gravity loads constant and reset time
# loadConst('-time', 0.0)


# --------------------------------
#       Perform eigen analysis
# --------------------------------

numEigen = 30
lameigenValues = eigen(numEigen)

wn = [i ** 0.5 for i in lameigenValues]
fn = [i / (2*math.pi) for i in wn]

print("Natural Frequencies:", fn, "Hz")

# compute the modal properties
#modalProperties("-print", "-file", "ModalReport.txt", "-unorm")

# -------------------------------------------
#       Plotting Structure and Modeshapes
# -------------------------------------------

# Display the active model with node and element tags
#plot_model("nodes","elements")

#plot_modeshape(1, 50)
#plot_modeshape(3, 50)
#plot_modeshape(6, 50)



# ----------------------------------
#       Define Sinusoidal Loading
# ----------------------------------

# Define parameters of the Sinusoidal Loading

T = 50 #Define how long the loading should go for
dt = 0.01 # Define the time step for input ground motion
tsteps = range(int((T/dt)+1))

# Define range of forcing frequency f (Hz)
f1 = 1/s
f2 = 5/s
df = 0.1 # Define discretization of forcing frequencies 
fsteps = range(int((f2-f1)/df))

# ddug [sin(w1*t1) sin(w2*t1)
#       sin(w1*t2) sin(w2*t2)
#       sin(w1*t3) sin(w2*t3)] Columns constant frequency, rows constant time

t = [] # Define time list
for i in (tsteps):
    t.append(dt*i)

dfddug = pd.DataFrame(index=[t]) # Define dataframe with all ddug data
    
# Create dataframe with all ground motion time series for all frequencies continuously
for i in fsteps:
    ddug = []
    for j in tsteps:
        y = (f1 + i*df)*(2*math.pi) # omega rad/s
#        print('omega'+str(y))
        x = math.sin(y*(dt*j)) # omega*t, sin reads radians
#        print('sin'+str(x))
#        print('inside'+str(y*(dt*j)))
#        print('time'+str(dt*j))
        ddug.append(x)
    dfddug['w' + str(i+1)] = ddug

dfddug.head(20)

# -------------------------------
#       Set up Dynamic analysis
# -------------------------------



f = []
peakVx = []
peakVy = []
peakMx = []
peakMy = []
dfVx = pd.DataFrame(index=[t]) # Define dataframe with all Base Shear x data
dfVy = pd.DataFrame(index=[t]) # Define dataframe with all Base Shear y data
dfMx = pd.DataFrame(index=[t]) # Define dataframe with all Overturning Moment about the x axis data
dfMy = pd.DataFrame(index=[t]) # Define dataframe with all Overturning Moment about the y axis data

# Define a function to return the maximum value of the steady state response
def eleMax(items, start = 0, end=None):
    return abs(max(items[start:end], key=abs))

# set the rayleigh damping factors for nodes & elements
rayleigh(0.0, 0.0, 0.0, 2*0.005/wn[0])

# Looping for each frequency
for i in fsteps:
    wipeAnalysis() # Wipe any previous pattern
    constraints('Plain')  			# how it handles boundary conditions
    numberer('Plain')
    system('BandGeneral')		    # how to store and solve the system of equations in the analysis
    algorithm('Newton')                 # use Newton-Raphson for linear analysis
    integrator('Newmark', 0.5, 0.25)        # create integration scheme, Newmark with alpha = 0.5, beta = 0.25
    analysis('Transient')      	    # define type of analysis static or transient

    # Create the convergence test, the norm of the residual with a tolerance of
    # 1e-12 and a max number of iterations of 10
    test('NormDispIncr', 1.0e-12,  10 )

# # # Run each ground motion per each frequency wf and extract the maximum moment per each sinusoidal ground motion
    f.append(f1 + (df*i)) # Keeping track of the frequencies
    Vx = [] # Resetting each vector for each frequency
    Vy = []
    Mx = []
    My = []
    setTime(0) # Reset time
    time = []
    
    # Setting up variables for the transient analysis
    tCurrent = getTime()
    tFinal = T
    ok = 0
    # Getting the correct ground motion per frequency step
    gacc = dfddug['w' + str(i+1)] # Ground motion for a constant wf, COMMENT OUT IF GROUND MOTIONS ALREADY EXIST
    gacc.to_csv('C:/Users/jxu20/Documents/groundmotions/'+'w'+str(i+1)+"ddug.csv", sep=',',index=False, header=False) # COMMENT OUT IF GROUND MOTIONS ALREADY EXIST
    # command: timeSeries('Path', timeIDTag, '-dt', timestep, '-factor', multiply load by g)
    timeSeries('Path', i+1, '-dt', dt, '-filePath', 'C:/Users/jxu20/Documents/groundmotions/'+'w'+str(i+1)+"ddug.csv", '-factor', g) 
    # patternTag, direction  accelTag
    pattern('UniformExcitation', i+1, 1, '-accel', i+1) # x direction

    # Perform the transient analysis
    while ok == 0 and tCurrent < tFinal:
        ok = analyze(1, dt)
        if ok != 0:
            print('WARNING' + str(tCurrent) + 'w' + str(i))
        # if the analysis fails try initial tangent iteration
#         if ok != 0:
#             print("regular newton failed .. let's try an initial stiffness for this step")
#             test('NormDispIncr', 1.0e-12,  100, 0)
#             algorithm('ModifiedNewton', '-initial')
#             ok = analyze(1, dt)
#             print("that worked .. back to regular newton")
#             test('NormDispIncr', 1.0e-12,  10 )
#             algorithm('Newton')
        # Store element forces per time step 
        tCurrent = getTime()
        time.append(tCurrent)     
        Vx.append(nodeReaction(1,1)) # Vx
        Vy.append(eleForce(1, 2)) # Vy
        Mx.append(eleForce(1, 4)) # Mx
        My.append(eleForce(1, 5)) # My
    # Post-processing
    maxVx = eleMax(Vx, int((T/dt)+1) - 30)
    peakVx.append(abs(max(Vx, key=abs)))
    peakVy.append(abs(max(Vy, key=abs)))
    peakMx.append(abs(max(Mx, key=abs)))
    peakMy.append(abs(max(My, key=abs)))
    dfVx['w'+str(i)] = Vx
    dfVy['w'+str(i)] = Vy
    dfMx['w'+str(i)] = Mx
    dfMy['w'+str(i)] = My
    
dfVy.head(20)

In [ ]:
#print(len(My))
#print(len(u1))
#MYTEST = pd.DataFrame(My)
#MYTEST.head(20)
#UTEST = pd.DataFrame(u3)
#UTEST.head(20)
print(peakVx)
print(f)





In [ ]:
#plt.plot(time, dfVx['w1'])
dfMy.head(30)

plt.plot(f, peakVx)
plt.ylabel('Peak Base Shear (kip)')
plt.xlabel('Frequency (Hz)')